In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
def get_listing_links(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm tất cả các thẻ <h3 class="post-title"> rồi trích xuất href từ thẻ <a>
    listings = soup.select("h3.post-title a")  # Lấy tất cả các thẻ <a> trong <h3 class="post-title">
    
    listing_links = []
    for listing in listings:
        link = listing['href']  # Lấy đường dẫn từ thuộc tính href
        full_link = f"https://phongtro123.com{link}"  # Thêm domain vào đường dẫn tương đối
        listing_links.append(full_link)
    
    return listing_links

In [4]:
def get_room_details(listing_url):
    response = requests.get(listing_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 1. Tiêu đề và rating
    title_tag = soup.select_one("h1.page-h1 a")
    title = title_tag.get_text() if title_tag else 'N/A'
    
    rating_tag = soup.select_one("h1.page-h1 .star")
    rating = rating_tag.get('class')[1].split('-')[1] if rating_tag else 'N/A'  # Lấy số lượng sao từ class star-5
    
    # 2. Địa chỉ
    address_tag = soup.select_one("address.post-address")
    address = address_tag.get_text().replace("Địa chỉ:", "").strip() if address_tag else 'N/A'

    # 3. Giá
    price_tag = soup.select_one("div.item.price span")
    price = price_tag.get_text() if price_tag else 'N/A'

    # 4. Diện tích
    area_tag = soup.select_one("div.item.acreage span")
    area = area_tag.get_text().strip() if area_tag else 'N/A'

    # 5. ID phòng trọ
    id_tag = soup.select_one("div.item.hashtag span")
    room_id = id_tag.get_text() if id_tag else 'N/A'

    # 6. Thời gian đăng
    published_tag = soup.select_one("div.item.published span")
    published_time = published_tag['title'] if published_tag and 'title' in published_tag.attrs else 'N/A'

    # 7. Mô tả
    description_tag = soup.select_one("section.section.post-main-content div.section-content")
    description = description_tag.get_text(separator="\n").strip() if description_tag else 'N/A'
    
    # 8. Ngày hết hạn
    expiration_date_tag = soup.select_one("table.table tr:nth-child(8) td time")
    expiration_date = expiration_date_tag.get_text() if expiration_date_tag else 'N/A'

    # 9. Loại tin rao (lấy nội dung trong cột thứ 2)
    listing_type_tag = soup.select_one("tr:contains('Loại tin rao:') td:nth-child(2)")
    listing_type = listing_type_tag.get_text().strip() if listing_type_tag else 'N/A'

    # 10. Đối tượng thuê (lấy nội dung trong cột thứ 2)
    tenant_type_tag = soup.select_one("tr:contains('Đối tượng thuê:') td:nth-child(2)")
    tenant_type = tenant_type_tag.get_text().strip() if tenant_type_tag else 'N/A'


    # Trả về dictionary chứa tất cả các thông tin
    return {
        'room_id': room_id,
        'title': title,
        'address': address,
        'price': price,
        'area': area,
        'published_time': published_time,
        'expiration_date': expiration_date,   
        'listing_type': listing_type,         
        'tenant_type': tenant_type,
        'description': description           
    }



In [5]:
def scrape_all_pages(base_url, num_pages):
    all_links = []
    for page in range(1, num_pages + 1):
        page_url = f"{base_url}?page={page}" if page > 1 else base_url  # Tạo URL từng trang
        listing_links = get_listing_links(page_url)
        all_links.extend(listing_links)
        print(f"Đã lấy xong dữ liệu từ trang {page}")  # In ra để kiểm tra tiến trình
    return all_links

In [6]:
def scrape_all_listings(base_url, num_pages):
    # Lấy tất cả các đường dẫn bài đăng
    all_links = scrape_all_pages(base_url, num_pages)

    # Trích xuất thông tin từ từng bài đăng
    all_room_details = []
    for link in all_links:
        room_details = get_room_details(link)
        all_room_details.append(room_details)
        
    df = pd.DataFrame(all_room_details)
    
    return all_room_details

In [7]:
base_url = "https://phongtro123.com/tinh-thanh/ho-chi-minh"  # Trang chính
num_pages = 550  # Số lượng trang bạn muốn scraping

In [8]:
# Scraping dữ liệu từ nhiều trang
room_data = scrape_all_listings(base_url, num_pages)

Đã lấy xong dữ liệu từ trang 1
Đã lấy xong dữ liệu từ trang 2
Đã lấy xong dữ liệu từ trang 3
Đã lấy xong dữ liệu từ trang 4
Đã lấy xong dữ liệu từ trang 5
Đã lấy xong dữ liệu từ trang 6
Đã lấy xong dữ liệu từ trang 7
Đã lấy xong dữ liệu từ trang 8
Đã lấy xong dữ liệu từ trang 9
Đã lấy xong dữ liệu từ trang 10
Đã lấy xong dữ liệu từ trang 11
Đã lấy xong dữ liệu từ trang 12
Đã lấy xong dữ liệu từ trang 13
Đã lấy xong dữ liệu từ trang 14
Đã lấy xong dữ liệu từ trang 15
Đã lấy xong dữ liệu từ trang 16
Đã lấy xong dữ liệu từ trang 17
Đã lấy xong dữ liệu từ trang 18
Đã lấy xong dữ liệu từ trang 19
Đã lấy xong dữ liệu từ trang 20
Đã lấy xong dữ liệu từ trang 21
Đã lấy xong dữ liệu từ trang 22
Đã lấy xong dữ liệu từ trang 23
Đã lấy xong dữ liệu từ trang 24
Đã lấy xong dữ liệu từ trang 25
Đã lấy xong dữ liệu từ trang 26
Đã lấy xong dữ liệu từ trang 27
Đã lấy xong dữ liệu từ trang 28
Đã lấy xong dữ liệu từ trang 29
Đã lấy xong dữ liệu từ trang 30
Đã lấy xong dữ liệu từ trang 31
Đã lấy xong dữ li

c:\Program Files\Python39\lib\site-packages\soupsieve\css_parser.py:862: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


In [9]:
df = pd.DataFrame(room_data)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   room_id          11000 non-null  object
 1   title            11000 non-null  object
 2   address          11000 non-null  object
 3   price            11000 non-null  object
 4   area             11000 non-null  object
 5   published_time   11000 non-null  object
 6   expiration_date  11000 non-null  object
 7   listing_type     11000 non-null  object
 8   tenant_type      11000 non-null  object
 9   description      11000 non-null  object
dtypes: object(10)
memory usage: 859.5+ KB


In [11]:
df.head()

,room_id,title,address,price,area,published_time,expiration_date,listing_type,tenant_type,description
0,649687,"GẦN TRƯỜNG GTVT, NGOẠI THƯƠNG, HUTECH, HỒNG BÀ...","97/15 Đường Ung Văn Khiêm, Phường 25, Quận Bìn...",4.5 triệu/tháng,30m2,"Thứ 6, 13:53 20/09/2024","Thứ 3, 10:30 24/09/2024","Phòng trọ, nhà trọ",Tất cả,- Để đảm bảo an ninh cho Sinh Viên ở Ngõ Sen g...
1,663051,KTX Giao Thông Vận Tải - Hutech 200m - 1.4 tri...,"69/38 Nguyễn Gia Trí, Phường 25, Quận Bình Thạ...",1.4 triệu/tháng,30m2,"Thứ 6, 00:46 20/09/2024","Thứ 6, 00:46 27/09/2024","Phòng trọ, nhà trọ",Tất cả,"Ký túc xá tiện nghi, cách ĐH Hutech chỉ 200m g..."
2,595869,Ký túc xá Tân Bình bao điện nước,"Đường Phạm Cự Lượng, Phường 2, Quận Tân Bình, ...",1.6 triệu/tháng,30m2,"Thứ 4, 20:24 18/09/2024","Thứ 7, 20:24 21/09/2024","Phòng trọ, nhà trọ",Tất cả,free 100% tiền phòng tháng đầu\nVị trí:\nGần c...
3,20126,Cho thuê phòng trọ dạng chung cư mini máy lạnh...,"Đường Hoàng Ngân, Phường 16, Quận 8, Hồ Chí Minh",1.8 triệu/tháng,18m2,"Thứ 4, 18:31 18/09/2024","Thứ 4, 18:31 25/09/2024","Phòng trọ, nhà trọ",Tất cả,"Cho thuê phòng trọ dạng chung cư mini P.16, Q...."
4,277630,Phòng trọ dạng căn hộ mini cao cấp tại 796 Lê ...,"796 Đường Lê Đức Thọ, Phường 15, Quận Gò Vấp, ...",3 triệu/tháng,30m2,"Thứ 3, 15:46 17/09/2024","Thứ 2, 09:38 07/10/2024","Phòng trọ, nhà trọ",Tất cả,Cho thuê căn hộ mini cao cấp tại 796 Lê Đức Th...


In [28]:
df["price"].value_counts()

price
3.5 triệu/tháng         814
4 triệu/tháng           787
4.5 triệu/tháng         724
3 triệu/tháng           596
3.8 triệu/tháng         488
                       ... 
4.25 triệu/tháng          1
4.77 triệu/tháng          1
5.55 triệu/tháng          1
2.999999 triệu/tháng      1
3.33 triệu/tháng          1
Name: count, Length: 193, dtype: int64